# Part 4: Almasru - examples
### Goals
* Learn to use Almasru library

### Short description
Almasru is a python library used to fetch easily records from SRU server. The system contains a lot of tools to find related records. The system make a backup of all requests. Unless you delete the history it will keep the results in memory.

The deduplication system is however not maintened anymore. Recommendation is to use the dedupmarcxml library: https://dedupmarcxml.readthedocs.io

### Documentation
* https://almasru.readthedocs.io
* https://github.com/Swiss-Library-Service-Platform/almasru

In [2]:
# Import libraries
from almasru.client import SruClient, SruRecord, IzSruRecord, SruRequest
from almasru.utils import check_removable_records, analyse_records
# from almasru import dedup
# from almasru.briefrecord import BriefRecFactory, BriefRec
from almasru import config_log

# Config logs
config_log()

In [4]:
# Fetch one record in NZ
SruClient.set_base_url('https://swisscovery.ch/view/sru/41SLSP_NETWORK')
record = SruRecord('991042095849705501')
print(record)

<record xmlns="http://www.loc.gov/MARC21/slim">
  <leader>00698naa a22001812c 4500</leader>
  <controlfield tag="001">991042095849705501</controlfield>
  <controlfield tag="005">20241004194331.0</controlfield>
  <controlfield tag="008">991211s1979    xxk           00| | eng d</controlfield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">(swissbib)032274408-41slsp_network</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">032274408</subfield>
    <subfield code="9">ExL</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">(IDSBB)001963059DSV01</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="040">
    <subfield code="a">SzZuIDS BS/BE A164</subfield>
    <subfield code="b">ger</subfield>
    <subfield code="e">kids</subfield>
    <subfield code="d">CH-ZuSLS</subfield>
  </datafield>
  <datafield ind1="1" ind2=" " tag="100">
    <subfield code="a">Chrisman, Richard</subfield>
  </da

In [6]:
parent_rec = record.get_parent_rec()

2025-09-02 00:49:01,711 - INFO - SRU data fetched: https://swisscovery.ch/view/sru/41SLSP_NETWORK?query=alma.other_system_number_active_035%3D%28IDSBB%29000010912DSV01&version=1.2&operation=searchRetrieve&startRecord=1&maximumRecords=50
2025-09-02 00:49:01,718 - INFO - Records 1 - 1 / 1, "alma.other_system_number_active_035=(IDSBB)000010912DSV01": 1


In [7]:
parent_rec

{'MMS_ID': '991042095849705501', 'related_records_found': True, 'number_of_rel_recs': 1, 'related_records': {SruRecord('991017428369705501', base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK')}, 'fields_related_records': [{'parent_MMS_ID': '991017428369705501', 'field': '773$w', 'content': '(IDSBB)000010912DSV01'}]}


In [10]:
# Find children records
child_records = SruRecord('991017428369705501').get_child_analytical_records()

In [12]:
# Make a query at NZ level
r = SruRequest(query='alma.isbn=9783030168773')
r.records

2025-09-02 00:58:05,006 - INFO - Records 1 - 2 / 2, "alma.isbn=9783030168773": 2


[SruRecord('991170263891705501', base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK'),
 SruRecord('991085109059705501', base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK')]

In [13]:
print(r.records[0])

<record xmlns="http://www.loc.gov/MARC21/slim">
  <leader>04043 am 222005773u 4500</leader>
  <controlfield tag="001">991170263891705501</controlfield>
  <controlfield tag="005">20230125193456.0</controlfield>
  <controlfield tag="006">m     o  d |</controlfield>
  <controlfield tag="007">cu#uuu---auuuu</controlfield>
  <controlfield tag="008">191030s2020    gw |    o    |||| 0|eng d</controlfield>
  <datafield ind1=" " ind2=" " tag="020">
    <subfield code="a">3-030-16877-8</subfield>
  </datafield>
  <datafield ind1="7" ind2=" " tag="024">
    <subfield code="a">10.1007/978-3-030-16877-3</subfield>
    <subfield code="2">doi</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">(CKB)4100000009678379</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">(OAPEN)1007055</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">(MiAaPQ)EBC6113652</subfield>
  </datafield>
  <datafi

In [18]:
# Make a query at IZ level
# "is_iz_request=True" is required to be able to fetch "IzSruRecord" instead of normal "SruRecord" objects
# "IzSruRecord" objects have additional methods to get information about inventory
SruClient.set_base_url('https://swisscovery.ch/view/sru/41SLSP_HES')
r = SruRequest(query='alma.isbn=2744009466', is_iz_request=True)
r.records 

2025-09-02 01:11:03,852 - INFO - Records 1 - 1 / 1, "alma.isbn=2744009466": 1


[IzSruRecord('990049508840205514', base_url='https://swisscovery.ch/view/sru/41SLSP_HES')]

In [19]:
record = r.records[0]

In [20]:
# This method provide information about availabable inventory
record.get_inventory_info()

2025-09-02 01:11:09,042 - INFO - IzSruRecord('990049508840205514', base_url='https://swisscovery.ch/view/sru/41SLSP_HES'): inventory in IZ found (1)


[{'IZ': '41SLSP_HES',
  'library': 'E41',
  'location': 'E41BI',
  'availability': 'available',
  'holding': '2289610230005514',
  'nb_items': '1'}]

In [21]:
print(record)

<record xmlns="http://www.loc.gov/MARC21/slim">
  <leader>00573nam a2200193 c 4500</leader>
  <controlfield tag="001">990049508840205514</controlfield>
  <controlfield tag="005">20201228175946.0</controlfield>
  <controlfield tag="008">050523s2000    fr            00| | fre^^</controlfield>
  <datafield ind1=" " ind2=" " tag="020">
    <subfield code="a">2744009466</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">(swissbib)161328075-41slsp_network</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">161328075</subfield>
    <subfield code="9">ExL</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">(NEBIS)004950884EBI01</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">(EXLNZ-41SLSP_NETWORK)991031277079705501</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="040">
    <subfield code="a">SzZuIDS NEBIS HES-EIG</subfield